In [9]:
import QuantLib as ql
import pandas as pd
from scipy.optimize import minimize

# Market data
spot_price = 176.65
strike_price = 190
market_price = 0.17  # Example market price of the option
risk_free_rate = 0.0525
dividend_yield = 0.0054
traditional_implied_volatility = 0.189461
calculation_date = ql.Date(3, 11, 2023)
maturity_date = ql.Date(24, 11, 2023)

def HestonParameters(spot_price, strike_price, market_price, dividend_yield, traditional_implied_volatility, calculation_date, maturity_date, risk_free_rate = .00525):
    day_count = ql.Actual365Fixed()
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)

    # Set up the QuantLib environment
    ql.Settings.instance().evaluationDate = calculation_date
    option_type = ql.Option.Call  # or ql.Option.Put for a put option
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    exercise = ql.EuropeanExercise(maturity_date)
    european_option = ql.VanillaOption(payoff, exercise)

    # Initial parameters for the Heston model
    initial_v0 = 0.1
    initial_kappa = 0.1
    initial_theta = traditional_implied_volatility
    initial_sigma = 0.1
    initial_rho = 0.1

    # Define the optimization objective function
    def objective_function(params):
        v0, kappa, theta, sigma, rho = params

        heston_process = ql.HestonProcess(
            ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count)),
            ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_yield, day_count)),
            ql.QuoteHandle(ql.SimpleQuote(spot_price)),
            v0, kappa, theta, sigma, rho
        )

        model = ql.HestonModel(heston_process)
        engine = ql.AnalyticHestonEngine(model)
        european_option.setPricingEngine(engine)

        model_price = european_option.NPV()
        error = (model_price - market_price) ** 2  # Squared error
        return error, model_price  # Return both error and the model price

    # Bounds for the parameters, excluding theta which is input by the user
    bounds = [(0.0001, 1.0), (0.0001, 2.0), (traditional_implied_volatility * 0.5, traditional_implied_volatility * 1.5), (0.0001, 1.0), (-0.999, 0.999)]

    # Initial parameter guesses, excluding theta which is input by the user
    initial_guess = [0.1, 0.1, traditional_implied_volatility, 0.1, 0.1]

    # Dictionary of optimizers to test, removing poor performers
    optimizers = {
        'L-BFGS-B': {'method': 'L-BFGS-B'},
        'TNC': {'method': 'TNC'},
    }

    # DataFrame to collect results
    results_df = pd.DataFrame(columns=['Optimizer', 'Success', 'Params', 'Objective_Value', 'Estimated_Price', 'Market_Price', 'MSE'])

    # Run optimizers and collect results
    for name, opt in optimizers.items():
        print(f"Running optimizer: {name}")
        
        # Run the optimizer and capture the additional returned value
        result = minimize(lambda x: objective_function(x)[0], initial_guess, method=opt['method'], bounds=bounds)
        
        success = result.success
        params = result.x if success else None
        # Now retrieve the full result from the objective function which includes the error and the model price
        objective_value, estimated_price = objective_function(params) if success else (None, None)
        error = (estimated_price - market_price) if success else None
        
        # Create a temporary DataFrame and concatenate it to the main DataFrame
        temp_df = pd.DataFrame({
            'Optimizer': [name],
            'Success': [success],
            'Params': [params],
            'Objective_Value': [objective_value],
            'Estimated_Price': [estimated_price],
            'Market_Price': [market_price if success else None],
            'MSE': [error**2]
        })
        results_df = pd.concat([results_df, temp_df], ignore_index=True)

    # Print the DataFrame
    results_df.sort_values(by='MSE', ascending= True)


Running optimizer: L-BFGS-B
Running optimizer: TNC


,Optimizer,Success,Params,Objective_Value,Estimated_Price,Market_Price,MSE
1,TNC,True,"[0.03128567186876707, 0.09915060841354195, 0.1...",4.429872e-15,0.170000,0.17,4.429872e-15
0,L-BFGS-B,True,"[0.0001, 0.0001, 0.0947305, 0.0001, -0.0532458...",2.742919e-02,0.004382,0.17,2.742919e-02
